In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os, re, argparse, csv
import shutil
from pprint import pprint

In [123]:
import _mypath
from bripipetools.io import parsers

In [10]:
dmplexstats_file = "../tests/test-data/genomics/Illumina/150615_D00565_0087_AC6VG0ANXX/Unaligned/DemultiplexingStats.xml"

In [126]:
dms_df = parsers.DemultiplexStatsParser(dmplexstats_file).parse_to_df()
dms_df

,Barcode,BarcodeCount,Flowcell,Lane,PerfectBarcodeCount,Project,Sample
0,unknown,3222151,C8B30ANXX,1,3222151,default,Undetermined
1,unknown,25352076,C8B30ANXX,2,25352076,default,Undetermined
2,unknown,25116427,C8B30ANXX,3,25116427,default,Undetermined
3,unknown,43836188,C8B30ANXX,4,43836188,default,Undetermined
4,unknown,44367529,C8B30ANXX,5,44367529,default,Undetermined
5,unknown,3398951,C8B30ANXX,6,3398951,default,Undetermined
6,unknown,3152223,C8B30ANXX,7,3152223,default,Undetermined
7,unknown,10183609,C8B30ANXX,8,10183609,default,Undetermined
8,ATCACG,8532450,C8B30ANXX,1,8420387,default,1_Agg225_3_B-1hc_rest
9,CGATGT,9432208,C8B30ANXX,1,9290929,default,2_Agg225_2_B-2RA_rest


In [6]:
# !pip install xmltodict
import xmltodict

In [11]:
with open(dmplexstats_file) as fd:
    doc = xmltodict.parse(fd.read())

In [16]:
doc['Stats']['Flowcell']

OrderedDict([(u'@flowcell-id', u'C8B30ANXX'),
             (u'Project',
              [OrderedDict([(u'@name', u'default'),
                            (u'Sample',
                             [OrderedDict([(u'@name', u'Undetermined'),
                                           (u'Barcode',
                                            [OrderedDict([(u'@name',
                                                           u'unknown'),
                                                          (u'Lane',
                                                           [OrderedDict([(u'@number',
                                                                          u'1'),
                                                                         (u'BarcodeCount',
                                                                          u'3222151'),
                                                                         (u'PerfectBarcodeCount',
                                                        

In [56]:
d = {'id': 'thing1',
     'project': {'name': 'foo',
                 'sample': {'name': 'bar',
                            'barcode': [{'name': 'baz',
                                        'lane': 1},
                                        {'name': 'baz2',
                                        'lane': 2}]}}}
print(d)
print('')

def flatten_xml_dict(d, key=None, d2=None, expand=1):
    if d2 is None:
        d2 = {}
    
    for k, v in d.items():
        if isinstance(v, list):
            for i in v:
                d2, m = flatten_xml_dict(i, k, d2)
        elif isinstance(v, dict):
            d2, m = flatten_xml_dict(v, k, d2)
        else:
            v = [v]
            if 'name' in k:
                k = '{}_{}'.format(key, k)
                v = v * 2
            for w in v:
                d2.setdefault(k, []).append(w)
    print d2
    m = max([len(i) for i in d2.values()])
    return (d2, m)

flatten_xml_dict(d['project'], 'project')
# print(x)

{'project': {'sample': {'barcode': [{'lane': 1, 'name': 'baz'}, {'lane': 2, 'name': 'baz2'}], 'name': 'bar'}, 'name': 'foo'}, 'id': 'thing1'}

{'lane': [1], 'barcode_name': ['baz', 'baz']}
{'lane': [1, 2], 'barcode_name': ['baz', 'baz', 'baz2', 'baz2']}
{'lane': [1, 2], 'barcode_name': ['baz', 'baz', 'baz2', 'baz2'], 'sample_name': ['bar', 'bar']}
{'lane': [1, 2], 'project_name': ['foo', 'foo'], 'barcode_name': ['baz', 'baz', 'baz2', 'baz2'], 'sample_name': ['bar', 'bar']}


({'barcode_name': ['baz', 'baz', 'baz2', 'baz2'],
  'lane': [1, 2],
  'project_name': ['foo', 'foo'],
  'sample_name': ['bar', 'bar']},
 4)

In [53]:
# !pip install pandas
x = []
x += ['a'] * 2
x

['a', 'a']

In [137]:
import collections
# dict2list = lambda dic: [(k, v) for (k, v) in dic.iteritems()]
name_not_all = lambda item: item['@name'] != 'all'

get_lane_details = lambda x: dict(x.items()) #{k: v for k, v in x.items()}
extract_fields = lambda x: {x['@name']: map(get_lane_details, x['Lane'])}

b = doc['Stats']['Flowcell']['Project'][0]['Sample'][0:1]
k = b[0].keys()
b[0][k[1]]
# f = filter(name_not_all, doc['Stats']['Flowcell']['Project'][0]['Sample'][0])


# map(lambda x: x, filter(name_not_all, b))

# pprint(doc['Stats']['Flowcell']['Project'][0]['Sample'][1]['Barcode'])

def unpack(item, key=None):
    unpack_dict = {}
    if isinstance(item, list):
        for i in item:
            unpack(item)
    if isinstance(item, dict):
        for k in item:
            unpack(item[k], k)
    else:
        print (key, item)
# print(b[0])
# print(isinstance(b[0], dict))
# print(b[0].keys())
# print(b[0][b[0].keys()[0]])
print(b[0][b[0].keys()[1]][0]['Lane'][0])

unpack(b[0][b[0].keys()[1]][0]['Lane'][0])

OrderedDict([(u'@number', u'1'), (u'BarcodeCount', u'3222151'), (u'PerfectBarcodeCount', u'3222151')])
(u'@number', u'1')
(u'BarcodeCount', u'3222151')
(u'PerfectBarcodeCount', u'3222151')


In [58]:
import xml.etree.ElementTree as ET
tree = ET.parse(dmplexstats_file)

In [59]:
tree

In [60]:
tree.getroot()

<Element 'Stats' at 0x10696a350>

In [122]:
root = tree.getroot()
dmux_dict = {}
for flowcell in list(root):
    for project in list(flowcell):
        for sample in list(project):
            for barcode in list(sample):
                if barcode.attrib['name'] != 'all':
                    for lane in list(barcode):
                        dmux_dict.setdefault(flowcell.tag, []).append(flowcell.attrib['flowcell-id'])
                        dmux_dict.setdefault(project.tag, []).append(project.attrib['name'])
                        dmux_dict.setdefault(sample.tag, []).append(sample.attrib['name'])
                        dmux_dict.setdefault(barcode.tag, []).append(barcode.attrib['name'])
                        dmux_dict.setdefault(lane.tag, []).append(lane.attrib['number'])

                        for count in list(lane)[0:2]:
                            dmux_dict.setdefault(count.tag, []).append(count.text)

pd.DataFrame(dmux_dict)

,Barcode,BarcodeCount,Flowcell,Lane,PerfectBarcodeCount,Project,Sample
0,unknown,3222151,C8B30ANXX,1,3222151,default,Undetermined
1,unknown,25352076,C8B30ANXX,2,25352076,default,Undetermined
2,unknown,25116427,C8B30ANXX,3,25116427,default,Undetermined
3,unknown,43836188,C8B30ANXX,4,43836188,default,Undetermined
4,unknown,44367529,C8B30ANXX,5,44367529,default,Undetermined
5,unknown,3398951,C8B30ANXX,6,3398951,default,Undetermined
6,unknown,3152223,C8B30ANXX,7,3152223,default,Undetermined
7,unknown,10183609,C8B30ANXX,8,10183609,default,Undetermined
8,ATCACG,8532450,C8B30ANXX,1,8420387,default,1_Agg225_3_B-1hc_rest
9,CGATGT,9432208,C8B30ANXX,1,9290929,default,2_Agg225_2_B-2RA_rest
